# Comprendiendo la Ecuación de Atención en la Arquitectura Transformer
Análisis de la inspiración detras de la Ecuación de Atención y sus componentes  en la Arquitectura Transformer, así como una implementación para su uso potencial en el modelo del lenguaje MIA.

## Instalación de libreria Numpy como pre-requisito
Ejecutar la siguiente celda para instalar la libreria numpy.

In [1]:
!pip install numpy

## Origen de los componentes que conforman la Ecuación de Atención
Los algoritmos de atención tomarón prestado los **conceptos de búsqueda de los Sistemas de Recuperación de Información** y lo empacaron como una **capa dentro de la Arquitectura Transformer**. El siguiente bloque de código muestra un tipo de búsqueda que encontramos conmunmente en muchos programas, busquedas exactas sobre la estructura de datos conocida como mapa. De igual forma muestra los principales componentes de este tipo de búsqueda:

*   **Q**(uery): La palabra utilizada como criterio de búsqueda.
*   **K**(keys): Un conjunto de llaves que forman parte de la estructura mapa y que mapea cada una de estas (1 a 1) con un valor correspondiente.
*   **V**(alues): Los valores que corresponden a cada llave en el mapa.

No es casual que la mátrices de pesos mencionadas en la Arquitectura Transformer utilicen las letras **Q, K, y V**.
Recordemos que la Arquitectura Transformer proviene de los laboratorios de Google y si existe una empresa que sabe de búsquedas es precisamente este coorporativo.

In [4]:
# Map of data, where normally exact search are executed in order to look for values
# associated with specific keys.

data = {
    "méxico": 7,
    "alemania": 10,
    "inglaterra": 8,
    "perro": 5
}

query = "méxico"
print(data[query])

print(data.keys())
print(data.values())

7
dict_keys(['méxico', 'alemania', 'inglaterra', 'perro'])
dict_values([7, 10, 8, 5])


## Ecuación de Atención y su relación con búsquedas por significado
Siguiendo la intuición detrás del algoritmo de atención, las búsquedas basados en el significado de la palabra consultada Q(uery) en lugar de búsquedas exactas son fundamentales para el algoritmo porque el resultado arrojara conceptos similares. El siguiente bloque de código muestra como podríamos realizar una búsqueda por significado utilizando **pesos (que tan importante es la llave K(eys) con respecto a la palabra utilizada como criterio Q(uery), la puntuación de atención)** y realizando una suma ponderada. Como seres humanos facilmente podemos ver que las 3 primeras llaves representan países y por tanto si buscamos por el concepto de "país" debemos estas llaves deben ser relevantes para los resultados que entreguemos (debemos prestar atención a las mismas) por lo que le **asignamos un peso de nuestra elección a cada una de ellas de acuerdo a que tan cerca esta la K(ey) con respecto a la palabra o Q(uery) consultada**.

In [5]:
# The intuition behind the attention mechanism defined in original transformer
# architecture is a search based in meaning instead of exact search taking advantage
# of dot product between vectors or matrix.
# This code only represent a weighted sum of attention scores (how similar we think)
# each key is from the provided query returning a value.

query = "país"
0.30 * data["méxico"] + 0.30 * data["alemania"] + 0.30 * data["inglaterra"] + 0.10 * data["perro"]

8.0

## Busquedas por significado y su relación con Embeddings
Los **Embeddings** de una palabra ***son vectores compactos con una dimensionalidad que son fundamentales en todo lo que se refiere al procesamiento de lenguaje natural incluidos los Modelos de Lenguaje Grandes y la arquitectura sobre la que se basan, la Arquitectura Transformer***. El siguiente bloque de código muestra un ejemplo de **búsqueda semántica** aprovechando la **Ecuación de Atención** definida en la Arquitectura Transformer; **esta es la ecuación utilizada para determinar la atención o dicho de otra manera los pesos que debemos asignar a cada combinación de palabras entre la consultada Q(uery) y los K(eys)** y que en la sección anterior elegimos de acuerdo a nuestra propia comprensión del lenguaje.

In [6]:
import numpy as np

def getWordEmbedding(word: str, embdim: int = 8):
    """
    Returns a normalized vector of provided dimension (by default 8) that emulate
    an embedding for the word. In a real implementation the word embedding should
    come from a real implementation.

    Parameters:
        word (string): Word to get embedding.
        embdim (int): Embedding dimesion, by default 8 when it is not provided.

    Returns:
        A vector that represent the embedding for the word.
    """
    return np.random.normal(size=(embdim))

def computeSoftmax(x):
    """
    Naive implementation of the softmax function using numpy library.

    Parameters:
        x (vector): Embedding vector that represent a word.

    Returns:
        A vector with normalized values for x.
    """
    return np.exp(x) / np.sum(np.exp(x))

def computeAttention(q, k, v):
    """
    Compute attention scores for query vector (an embedding vector).

    Parameters:
        q (vector): Embedding vector that represent a word.
        keys (list): A list of keys being compared with the query vector based
        in its similarity. Internally the transpose of this parameter is being used.
        values (list): Each value being mapped by the keys.

    Returns:
        A matrix with all the computed attention scores.
    """

    return computeSoftmax(q @ k.T) @ v

def searchByMeaning(queryWord: str, keys, values):
    """
    Search for the query word by its meaning using the concept of embeddings similarity
    result of a dot product and assigning a weight of attention to each combination of
    the queried word with provided keys.

    Parameters:
        queryword (string): Queried word. Internally an embeding of it is obtained
        before to evaluate the attention equation.
        keys (list): A list of keys being compared with the queryword based in its similarity. Internally
        keys will be a matrix of stacked embeddings of each provided word.
        values (list): Each value being mapped by the keys.

    Returns:
        A matrix with all the computed attention scores.
    """
    q = getWordEmbedding(queryWord)
    k = np.array([getWordEmbedding(key) for key in keys])
    v = values

    attentionScore = computeAttention(q, k, v)

    return attentionScore

print(searchByMeaning("país", ["méxico", "alemania", "inglaterra", "perro"], [10, 5, 2, 4]))

2.1743777856094124


## Implementación de Ecuación de Atención con Numpy
El siguiente bloque de código es una **implementación completa de la Ecuación de Atención** descrita en el documento que define la Arquitectura Transformer aprovechando la libreria numpy, como puede verse el código es realmente sencillo y basicamente queda totalmente expresada utilizando la siguiente linea de código.

`softmax(Q @ K.T / np.sqrt(dimension)) @ V`

In [ ]:
import numpy as np

def softmax(x):
    """
    Implementation of softmax equation with numpy library.

    Parameters:
        x (matrix): A matrix.

    Returns:
        Normalized matrix, any element in the matrix is real value in following
        range 0<= item <=1.
    """
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)

def attention(Q, K, V):
    """
    Computes the attention score, evaluate the scaled dot product attention equation
    computing the dot products of the Q(uery) with all the K(eys) transpose, divide each by
    sqrt(dimension) then apply a softmax function to obtain normalized weights.

    Parameters:
        Q (matrix): Q(uery) matrix, basically a matrix where each row is the
        embedding being analyzed.
        K (matrix): K(eys) matrix, same matrix than Q however the transpose is used
        in order to compute a dod product where word is compared each other for similarity
        taking advantage of dot product between matrix.
        V (matrix): V(values) from the attention equation
    """

    # It is expected that Q, K, and V has the same dimentionality, however,  we
    # are not enforcing it with any validation.
    dimension = K.shape[-1]

    return softmax(Q @ K.T / np.sqrt(dimension)) @ V